In [ ]:
import numpy as np
import open3d as o3d

frame = []

# Load the numpy file
tf_0 = np.load(r'TestingData\view00_tf.npy')
tf_1 = np.load(r'TestingData\view01_tf.npy')
tf_2 = np.load(r'TestingData\view02_tf.npy')
tf_3 = np.load(r'TestingData\view03_tf.npy')
tf_4 = np.load(r'TestingData\view04_tf.npy')
tf_obj = np.load(r'TestingData\initial_POS.npy')

# Load pcd files
pcd_0 = o3d.io.read_point_cloud(r"TestingData\view00.pcd")
pcd_1 = o3d.io.read_point_cloud(r"TestingData\view01.pcd")
pcd_2 = o3d.io.read_point_cloud(r"TestingData\view02.pcd")
pcd_3 = o3d.io.read_point_cloud(r"TestingData\view03.pcd")
pcd_4 = o3d.io.read_point_cloud(r"TestingData\view04.pcd")


world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[0, 0, 0])
world_frame_x = o3d.geometry.TriangleMesh.create_coordinate_frame(size=75.0, origin=[20, 0, 0])
world_frame_y = o3d.geometry.TriangleMesh.create_coordinate_frame(size=24.0, origin=[0, 20, 0])
frame.append(world_frame)
frame.append(world_frame_x)
frame.append(world_frame_y)

# tf_0_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_0_frame.transform(tf_0)
# tf_1_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_1_frame.transform(tf_1)
# tf_2_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_2_frame.transform(tf_2)
# tf_3_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_3_frame.transform(tf_3)
# tf_4_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0)
# tf_4_frame.transform(tf_4)
# frame.append(tf_0_frame)
# frame.append(tf_1_frame)
# frame.append(tf_2_frame)
# frame.append(tf_3_frame)
# frame.append(tf_4_frame)

# object_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=tf_obj)
# frame.append(object_frame)


# o3d.visualization.draw_geometries(frame + [pcd_0])
o3d.visualization.draw_geometries(frame + [pcd_0, pcd_1, pcd_2, pcd_3, pcd_4])



In [ ]:
import open3d as o3d
import numpy as np
import math

def rotate_pcd_z_180(input_path, output_path):
    # 1. Load the PCD file
    pcd = o3d.io.read_point_cloud(input_path)
    if pcd.is_empty():
        print("Failed to load file. Please check the path.")
        return

    # 2. Set rotation angle (in radians)
    # 180 degrees is equal to np.pi.
    radian = math.radians(90)

    # 3. Create a rotation matrix based on the Z-axis
    # It is convenient to use get_rotation_matrix_from_axis_angle.
    R = pcd.get_rotation_matrix_from_axis_angle([0, 0, radian])

    # 4. Apply rotation
    # Adding center=pcd.get_center() allows rotation around the center of the data.
    pcd.rotate(R, center=(0, 0, 0))

    # 5. Save the result
    o3d.io.write_point_cloud(output_path, pcd)
    print(f"Rotated file has been saved: {output_path}")

# Usage Example
rotate_pcd_z_180("workpiece/z_bracket_10000.pcd", "workpiece/z_bracket_10000_rotated.pcd")

# Visualize the original and rotated point clouds
world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[0, 0, 0])
original_pcd = o3d.io.read_point_cloud("workpiece/z_bracket_10000.pcd")
rotated_pcd = o3d.io.read_point_cloud("workpiece/z_bracket_10000_rotated.pcd")

o3d.visualization.draw_geometries([world_frame, original_pcd, rotated_pcd])

Rotated file has been saved: workpiece/z_bracket_10000_rotated.pcd


In [ ]:
print("A")

array([[-1.18627030e-04, -9.06231562e-01, -4.22781672e-01,
         8.67716304e+02],
       [-9.99999914e-01,  2.75216927e-04, -3.09339914e-04,
         1.05647982e+02],
       [ 3.96690265e-04,  4.22781599e-01, -9.06231517e-01,
         3.56807697e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
import math
import os 

def merge_pcds_with_boundary(data_dir, initial_POS_YOLO, extent, initial_angle):
    """
    다각도에서 찍은 Point Cloud를 Bounding 박스로 자르고 하나로 합치는 함수

    Returns:
        Point cloud,  Bounding box
    """
    pcd_files = [f for f in os.listdir(data_dir) if f.endswith('.pcd') and "view0" in f]
    pcd_files.sort()
    merged_pcd = o3d.geometry.PointCloud()

    # OBB Box 회전 각도 확인 필요
    rotation_matrix = get_rotation_matrix_z(-initial_angle)
    # print("rotation matrix: ", rotation_matrix)
    center = np.array(initial_POS_YOLO)
    extent = np.array(extent)
    
    obb = o3d.geometry.OrientedBoundingBox(center, rotation_matrix, extent)
    obb.color = (1, 0, 0)

    for file_name in pcd_files:
        file_path = os.path.join(data_dir, file_name)
        pcd = o3d.io.read_point_cloud(file_path)
        #지면 제거
        plane_model, inliers = pcd.segment_plane(distance_threshold = 3.0, ransac_n = 3, num_iterations = 2000)
        [a, b, c, d] = plane_model
        pts = np.asarray(pcd.points)
        distances = a * pts[:, 0] + b * pts[:, 1] + c * pts[:, 2] + d
        removed_ground = np.where(abs(distances) > 1.5)[0]
        pcd = pcd.select_by_index(removed_ground)

        # BoundaryBox 제거
        pcd = pcd.crop(obb)
        
        merged_pcd += pcd
        print(f"Added {file_name}: {len(pcd.points)} Pointcloud Data being Merged")
        # coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100, origin=[0, 0, 0])
        # o3d.visualization.draw_geometries([merged_pcd, coord])

    # if not merged_pcd.is_empty():
    #     robot_base = o3d.geometry.TriangleMesh.create_coordinate_frame(size=200.0)
    #     o3d.visualization.draw_geometries([merged_pcd, robot_base, obb], 
    #                                         window_name="Merged and Cropped PCD")  
        
    # BoundaryBox 제거
    # merged_pcd = merged_pcd.crop(obb)
    o3d.visualization.draw_geometries([merged_pcd, obb])
    return merged_pcd, obb

def get_rotation_matrix_z(deg):
    """
    입력된 각도(deg)만큼 z축으로 회전하는 행렬을 생성하는 함수

    Returns:
        np.ndarray: z축 회전 행렬 (shape: (3,3))
    """
    rad = math.radians(deg)
    c = math.cos(rad)
    s = math.sin(rad)
    return np.array([
        [c, -s, 0],
        [s,  c, 0],
        [0,  0, 1]
    ])


initial_POS = [609.1741819176123, 73.19215154987111, 3.503121243592659]
initial_angle = 88.0

box_size = [100, 100, 80]
# box_size = [65, 85, 60]
# box_size = [1100, 1100, 1000]
pcd_data_dir = "PCD_Data"

merged_pcd, boundary_box = merge_pcds_with_boundary(pcd_data_dir, initial_POS, box_size, initial_angle)
if not merged_pcd.is_empty():
    o3d.io.write_point_cloud("PCD_Data/1_multiview_merged.pcd", merged_pcd)
else:
    print("No Merged PCD")

Added view00.pcd: 7277 Pointcloud Data being Merged
Added view01.pcd: 8702 Pointcloud Data being Merged
Added view02.pcd: 6273 Pointcloud Data being Merged
Added view03.pcd: 5374 Pointcloud Data being Merged
Added view04.pcd: 7253 Pointcloud Data being Merged


In [ ]:
def preprocess_point_cloud_fps(pcd, num_points):
    """
    Function to obtain Downsampling, Normal estimation, and FPFH for Ransac from a Point Cloud.

    Returns:
        Point Cloud (Downsampled), FPFH
    """
    # Farthest Point Sampling (FPS) to reduce the number of points
    pcd_fps = pcd.farthest_point_down_sample(num_points)
    
    # Calculate average distance between points to determine search radius
    avg_dist = np.mean(pcd_fps.compute_nearest_neighbor_distance())
    
    radius_normal = avg_dist * 2
    # Estimate surface normals for each point
    pcd_fps.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    
    # Align normal vectors to the Z-axis
    normals = np.asarray(pcd_fps.normals)
    for i in range(len(normals)):
        if normals[i][2] < 0:   
            normals[i] *= -1

    radius_feature = avg_dist * 5
    # Compute Fast Point Feature Histograms (FPFH) for global registration
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd_fps, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

    return pcd_fps, pcd_fpfh

print("Downsampling in Process..")
source = o3d.io.read_point_cloud("workpiece/z_bracket_10000.pcd")
target = o3d.io.read_point_cloud("PCD_Data/1_multiview_merged.pcd")
source.paint_uniform_color([1.0, 0.0, 0.0])
num_points = 10000
num_points_target = 10000

target_save_path = "PCD_Data/2_target_fps_downsampled.pcd"
# if os.path.exists(target_save_path):
if False:
    print(f"기존에 저장된 파일을 불러옵니다: {target_save_path}")
    target_down = o3d.io.read_point_cloud(target_save_path)
    avg_dist = np.mean(target_down.compute_nearest_neighbor_distance())
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=avg_dist * 2, max_nn=30))
    target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        target_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=avg_dist * 5, max_nn=100))
else:
    target_down, target_fpfh = preprocess_point_cloud_fps(target, num_points_target)
    o3d.io.write_point_cloud(target_save_path, target_down)
    print(f"Operation completed and saved: {target_save_path}")

source_down, source_fpfh = preprocess_point_cloud_fps(source, num_points)

Downsampling in Process..
Operation completed and saved: PCD_Data/2_target_fps_downsampled.pcd


In [63]:
import copy

def RANSAC(source_down, target_down, source_fpfh, target_fpfh):
    max_attempts = 5
    attempt = 0
    results_history = []
    best_fitness = -0.1
    best_inlier_rmse = 100.0
    best_result = None
    best_threshold = None 

    while attempt < max_attempts:
        attempt += 1
        # thresholds = [10.0, 8.0, 6.0, 4.0, 2.0]
        # thresholds = [10.0, 9.0, 8.0, 7.0, 6.0]
        thresholds = [10.0, 9.5, 9.0, 8.5, 8.0, 7.5, 7.0, 6.5, 6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 3.0]
        
        for thr in thresholds:            
            result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
                source_down, target_down, source_fpfh, target_fpfh, True,
                thr,
                o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
                3, [
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.85),
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(thr)
                ], o3d.pipelines.registration.RANSACConvergenceCriteria(10000, 0.99)
            )
            results_history.append({
                'threshold': thr,
                'fitness': result.fitness,
                'rmse': result.inlier_rmse
            })
            # temp_source = copy.deepcopy(source)
            # temp_source.transform(result.transformation)
            # o3d.visualization.draw_geometries([temp_source, target_down], window_name="RANSAC Result")

            if result.fitness > 0.85 and result.inlier_rmse < best_inlier_rmse:
                best_fitness = result.fitness
                best_inlier_rmse = result.inlier_rmse
                best_result = result
                best_threshold = thr
            if best_fitness > 0.95 and best_inlier_rmse < 2.35: 
                return best_result, best_threshold
            
        for res in results_history:
            marker = ">>" if res['fitness'] == best_fitness and res['rmse'] == best_inlier_rmse else "  "
            print(f"{marker} {res['threshold']:<9.2f} | {res['fitness']:<12.4f} | {res['rmse']:<12.4f}")
        print("="*55)
        print(f"Final Selection -> Fitness: {best_fitness:.4f}, RMSE: {best_inlier_rmse:.4f}")

    # if best_fitness > 0.90 and best_inlier_rmse < 2.4: 
    return best_result, best_threshold


print("RANSAC Process (Global Fitting)")
target_down.paint_uniform_color([0, 0.651, 0.929])
result_ransac, best_thr = RANSAC(source_down, target_down, source_fpfh, target_fpfh)

source_temp = copy.deepcopy(source) 
source_temp.transform(result_ransac.transformation)
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100, origin=[0, 0, 0])
o3d.visualization.draw_geometries([source_temp, target_down,coord], window_name="RANSAC Result")

RANSAC Process (Global Fitting)
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open3D WARNING] Too few correspondences (199) after mutual filter, fall back to original correspondences.
[Open

In [65]:
def execute_icp_registration(source, target, transformation, best_threshold):
    # thresholds = [10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]
    multipliers = [1.0, 0.8, 0.5, 0.25]
    thresholds = [best_threshold * m for m in multipliers]
    best_fitness = -0.1
    best_inlier_rmse = 100.0
    results_history = []

    # voxel_size = 0.5
    # target.estimate_normals(
    #     o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    # source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

    current_transformation = transformation

    for thr in thresholds:
        reg_p2l = o3d.pipelines.registration.registration_icp(
            source, target, thr, current_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(), #PointToPlane
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration = 1000) 
        )
        results_history.append({
            'threshold': thr,
            'fitness': reg_p2l.fitness,
            'rmse': reg_p2l.inlier_rmse
        })

        if reg_p2l.fitness > 0.85 and reg_p2l.inlier_rmse < best_inlier_rmse:
            best_fitness = reg_p2l.fitness
            best_inlier_rmse = reg_p2l.inlier_rmse
            best_result = reg_p2l

    for res in results_history:
        marker = ">>" if res['fitness'] == best_fitness and res['rmse'] == best_inlier_rmse else "  "
        print(f"{marker} {res['threshold']:<9.2f} | {res['fitness']:<12.4f} | {res['rmse']:<12.4f}")
    print("="*55)
    print(f"Final Selection -> Fitness: {best_fitness:.4f}, RMSE: {best_inlier_rmse:.4f}")

    return best_result

print("ICP Pose Estimation")
initial_trans = result_ransac.transformation
result_icp = execute_icp_registration(source, target_down, initial_trans, best_thr)

source_final = copy.deepcopy(source)
source_final.transform(result_icp.transformation)

source_final.paint_uniform_color([1, 0, 0])
target_down.paint_uniform_color([0, 0.651, 0.929])

o3d.visualization.draw_geometries([source_final, target_down, coord], window_name="Final ICP Result")

initial_POS = result_icp.transformation[:3, 3]
print("initial POS", initial_POS)
initial_ORI = result_icp.transformation[:3, :3]
print("initial Ori", initial_ORI)


ICP Pose Estimation
   8.00      | 0.9416       | 2.6493      
>> 6.40      | 0.9067       | 2.3041      
   4.00      | 0.7044       | 1.7842      
   2.00      | 0.5347       | 1.2757      
Final Selection -> Fitness: 0.9067, RMSE: 2.3041
initial POS [595.99105194  82.07240567  -3.90693426]
initial Ori [[-0.99660191  0.08227039 -0.00402748]
 [-0.08232484 -0.99647654  0.01603499]
 [-0.00269408  0.01631207  0.99986332]]


[[ 0.99791494 -0.03700979 -0.05287773]
 [ 0.03742708  0.99927538  0.00692295]
 [ 0.0525832  -0.00888758  0.998577  ]]


In [12]:
import open3d as o3d
import numpy as np
import math
import copy
import os


def get_rotation_matrix_z(deg):
    """Creates a 3x3 rotation matrix for the Z-axis."""
    rad = math.radians(deg)
    c, s = math.cos(rad), math.sin(rad)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])


def merge_multiview_scan(data_dir, initial_pos, initial_angle, box_size):
    """Merges multiple view PCDs into one target cloud within a bounding box."""
    pcd_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.pcd') and "view0" in f])
    merged_pcd = o3d.geometry.PointCloud()
    
    # Setup Oriented Bounding Box (OBB) for cropping
    obb = o3d.geometry.OrientedBoundingBox(
        np.array(initial_pos), 
        get_rotation_matrix_z(-initial_angle), 
        np.array(box_size)
    )

    for file_name in pcd_files:
        pcd = o3d.io.read_point_cloud(os.path.join(data_dir, file_name))
        
        # Remove ground plane
        _, inliers = pcd.segment_plane(distance_threshold=3.0, ransac_n=3, num_iterations=2000)
        pcd = pcd.select_by_index(inliers, invert=True) 
        
        # Crop to the area where the object is expected
        merged_pcd += pcd.crop(obb)
        
    return merged_pcd


def preprocess_point_cloud(pcd, num_points):
    """Downsamples, estimates normals, and computes FPFH features."""
    pcd_down = pcd.farthest_point_down_sample(num_points)
    avg_dist = np.mean(pcd_down.compute_nearest_neighbor_distance())
    
    # Estimate Normals
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=avg_dist * 2, max_nn=30))
    
    # Orientation fix: Ensure normals point 'up' (positive Z)
    normals = np.asarray(pcd_down.normals)
    for i in range(len(normals)):
        if normals[i][2] < 0:
            normals[i] *= -1

    # Compute FPFH (Feature descriptors for global matching)
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down, o3d.geometry.KDTreeSearchParamHybrid(radius=avg_dist * 5, max_nn=100))
    
    return pcd_down, fpfh


def run_global_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    """
    Performs RANSAC-based global registration to find a rough alignment.
    """
    distance_threshold = voxel_size * 1.5
    
    # RANSAC based on feature matching
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh, 
        mutual_filter=True,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=3, 
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ], 
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
    )
    return result


def run_local_refinement(source, target, initial_transformation, voxel_size):
    """
    Performs ICP registration to refine the alignment found by RANSAC.
    """
    # We use a smaller threshold for ICP to ensure high precision
    distance_threshold = voxel_size * 0.4
    
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, initial_transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
    )
    return result

In [26]:
# --- 1. Configuration ---
DATA_DIR = "PCD_Data"
SOURCE_PATH = "workpiece/z_bracket_10000.pcd" #  CAD model
YOLO_POS = [609.17, 73.19, 3.50]              # Initial guess from YOLO
YOLO_ANGLE = 88.0                             # Initial angle from YOLO
CROP_BOX = [100, 100, 80]                     # Region of interest size
VOXEL_SIZE = 5.0                              # Base unit for search distance


# --- 2. Data Preparation ---
print("Step 1: Merging and cleaning scans...")
source_cloud = o3d.io.read_point_cloud(SOURCE_PATH)
target_cloud = merge_multiview_scan(DATA_DIR, YOLO_POS, YOLO_ANGLE, CROP_BOX)
o3d.visualization.draw_geometries([target_cloud], window_name="Merged Target Cloud")

# Get downsampled versions and features for RANSAC
source_down, source_fpfh = preprocess_point_cloud(source_cloud, 10000)
target_down, target_fpfh = preprocess_point_cloud(target_cloud, 10000)
o3d.visualization.draw_geometries([target_down], window_name="Downsampled Target Cloud")


# --- 3. Global Alignment (RANSAC) ---
print("Step 2: Running RANSAC Global Registration...")
ransac_res = run_global_registration(source_down, target_down, source_fpfh, target_fpfh, VOXEL_SIZE)

# Visualize RANSAC result
source_temp = copy.deepcopy(source_cloud)
source_temp.transform(ransac_res.transformation)
source_temp.paint_uniform_color([1, 0, 0])
target_down.paint_uniform_color([0, 0.651, 0.929])
o3d.visualization.draw_geometries([source_temp, target_down], window_name="RANSAC Result")


# --- 4. Local Alignment (ICP) ---
print("Step 3: Running ICP Local Refinement...")
icp_res = run_local_refinement(source_cloud, target_down, ransac_res.transformation, VOXEL_SIZE)


Step 1: Merging and cleaning scans...
Step 2: Running RANSAC Global Registration...
[Open3D WARNING] Too few correspondences (194) after mutual filter, fall back to original correspondences.
Step 3: Running ICP Local Refinement...


In [27]:
ransac_res

RegistrationResult with fitness=9.246000e-01, inlier_rmse=3.271769e+00, and correspondence_set size of 9246
Access transformation to get result.

In [25]:
# --- 5. Extract Final Results ---
final_matrix = icp_res.transformation
initial_POS = final_matrix[:3, 3] #
initial_ORI = final_matrix[:3, :3] #
print("="*30)
print(f"Final Position: {initial_POS}")
print(f"Final Orientation Matrix:\n{initial_ORI}")
print(f"Fitness: {icp_res.fitness:.4f}")
print(f"RMSE: {icp_res.inlier_rmse:.4f}")


# --- 6. Final Visualization ---
source_final = copy.deepcopy(source_cloud).transform(final_matrix)
source_final.paint_uniform_color([1, 0, 0])        # Red: CAD Model
target_cloud.paint_uniform_color([0, 0.65, 0.93]) # Blue: Scanned Data
o3d.visualization.draw_geometries([source_final, target_cloud])

Final Position: [610.04845327  66.21433591  -4.24642886]
Final Orientation Matrix:
[[ 0.1870216  -0.9823343   0.0064992 ]
 [ 0.96764387  0.18535741  0.17119571]
 [-0.1693761  -0.02572839  0.9852156 ]]
Fitness: 0.6179
RMSE: 1.2200
